In [8]:
import pandas as pd
from ctgan import CTGAN
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp
import numpy as np

# Load the dataset
data = pd.read_csv('diabetes.csv')

# Separate features and target
features = data.drop(columns=['Outcome'])
target = data['Outcome']

# Normalize the features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

# Combine normalized features with target for CTGAN
normalized_data = pd.DataFrame(normalized_features, columns=features.columns)
normalized_data['Outcome'] = target.values

# Initialize and train CTGAN
ctgan = CTGAN()
ctgan.fit(normalized_data, epochs=1000)

# Sample synthetic data
num_samples = len(data)
synthetic_data = ctgan.sample(10000)

# Inverse transform the normalized features
synthetic_features = synthetic_data.drop(columns=['Outcome'])
synthetic_features = scaler.inverse_transform(synthetic_features)
synthetic_data[features.columns] = synthetic_features

# Convert synthetic data to the same format as original data
synthetic_data = synthetic_data[normalized_data.columns]

# Statistical comparison
def compare_statistics(original, synthetic):
    stats = {
        "mean": {},
        "std": {},
        "min": {},
        "max": {},
    }

    for column in original.columns:
        orig_stats = original[column].describe()
        synth_stats = synthetic[column].describe()

        stats["mean"][column] = (orig_stats["mean"], synth_stats["mean"])
        stats["std"][column] = (orig_stats["std"], synth_stats["std"])
        stats["min"][column] = (orig_stats["min"], synth_stats["min"])
        stats["max"][column] = (orig_stats["max"], synth_stats["max"])

    return stats

def percent_difference(orig_val, synth_val):
    return abs(orig_val - synth_val) / orig_val * 100

# Compute statistics
stats = compare_statistics(data, synthetic_data)

# Print results and percent differences
for stat_type, columns in stats.items():
    print(f"\n{stat_type.upper()}:")
    for column, values in columns.items():
        orig_val, synth_val = values
        diff = percent_difference(orig_val, synth_val)
        print(f"{column} - Original: {orig_val:.4f}, Synthetic: {synth_val:.4f}, Percent Difference: {diff:.2f}%")

print("\nKOLMOGOROV-SMIRNOV TEST:")
for column in features.columns:
    stat, p_value = ks_2samp(data[column], synthetic_data[column])
    print(f"{column} - K-S Statistic: {stat:.4f}, P-value: {p_value:.4f}")



MEAN:
Pregnancies - Original: 3.8451, Synthetic: 2.9332, Percent Difference: 23.72%
Glucose - Original: 120.8945, Synthetic: 107.6811, Percent Difference: 10.93%
BloodPressure - Original: 69.1055, Synthetic: 65.4201, Percent Difference: 5.33%
SkinThickness - Original: 20.5365, Synthetic: 15.0024, Percent Difference: 26.95%
Insulin - Original: 79.7995, Synthetic: 70.3147, Percent Difference: 11.89%
BMI - Original: 31.9926, Synthetic: 30.4713, Percent Difference: 4.76%
DiabetesPedigreeFunction - Original: 0.4719, Synthetic: 0.2709, Percent Difference: 42.60%
Age - Original: 33.2409, Synthetic: 39.2300, Percent Difference: 18.02%
Outcome - Original: 0.3490, Synthetic: 0.1824, Percent Difference: 47.73%

STD:
Pregnancies - Original: 3.3696, Synthetic: 3.5410, Percent Difference: 5.09%
Glucose - Original: 31.9726, Synthetic: 31.1526, Percent Difference: 2.56%
BloodPressure - Original: 19.3558, Synthetic: 20.8618, Percent Difference: 7.78%
SkinThickness - Original: 15.9522, Synthetic: 17.05

/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/3285819046.py:59: RuntimeWarning: divide by zero encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100
/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/3285819046.py:59: RuntimeWarning: invalid value encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100


In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp

# Load the dataset
data = pd.read_csv('diabetes.csv')

# Separate features and target
features = data.drop(columns=['Outcome'])
target = data['Outcome']

# Normalize the features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

# Convert to torch tensors
X = torch.tensor(normalized_features, dtype=torch.float32)
y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)

# Define the neural network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.network(x)

input_dim = X.shape[1]
output_dim = X.shape[1]

generator = Generator(input_dim, output_dim)

# Define the optimizer and loss function
optimizer = optim.Adam(generator.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Train the neural network
num_epochs = 1000
batch_size = 64

for epoch in range(num_epochs):
    permutation = torch.randperm(X.size()[0])
    for i in range(0, X.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X[indices], y[indices]

        outputs = generator(batch_x)
        loss = criterion(outputs, batch_x)
        loss.backward()
        optimizer.step()

# Sample synthetic data
with torch.no_grad():
    synthetic_data = generator(X).numpy()

# Inverse transform the normalized features
synthetic_features = scaler.inverse_transform(synthetic_data)
synthetic_df = pd.DataFrame(synthetic_features, columns=features.columns)
synthetic_df['Outcome'] = target.values

# Statistical comparison
def compare_statistics(original, synthetic):
    stats = {
        "mean": {},
        "std": {},
        "min": {},
        "max": {},
    }

    for column in original.columns:
        orig_stats = original[column].describe()
        synth_stats = synthetic[column].describe()

        stats["mean"][column] = (orig_stats["mean"], synth_stats["mean"])
        stats["std"][column] = (orig_stats["std"], synth_stats["std"])
        stats["min"][column] = (orig_stats["min"], synth_stats["min"])
        stats["max"][column] = (orig_stats["max"], synth_stats["max"])

    return stats

def percent_difference(orig_val, synth_val):
    return abs(orig_val - synth_val) / orig_val * 100

# Compute statistics
stats = compare_statistics(data, synthetic_df)

# Print results and percent differences
for stat_type, columns in stats.items():
    print(f"\n{stat_type.upper()}:")
    for column, values in columns.items():
        orig_val, synth_val = values
        diff = percent_difference(orig_val, synth_val)
        print(f"{column} - Original: {orig_val:.4f}, Synthetic: {synth_val:.4f}, Percent Difference: {diff:.2f}%")

# Kolmogorov-Smirnov Test
print("\nKOLMOGOROV-SMIRNOV TEST:")
for column in features.columns:
    stat, p_value = ks_2samp(data[column], synthetic_df[column])
    print(f"{column} - K-S Statistic: {stat:.4f}, P-value: {p_value:.4f}")



MEAN:
Pregnancies - Original: 3.8451, Synthetic: 3.8466, Percent Difference: 0.04%
Glucose - Original: 120.8945, Synthetic: 121.3848, Percent Difference: 0.41%
BloodPressure - Original: 69.1055, Synthetic: 69.0705, Percent Difference: 0.05%
SkinThickness - Original: 20.5365, Synthetic: 20.6119, Percent Difference: 0.37%
Insulin - Original: 79.7995, Synthetic: 80.6122, Percent Difference: 1.02%
BMI - Original: 31.9926, Synthetic: 31.9545, Percent Difference: 0.12%
DiabetesPedigreeFunction - Original: 0.4719, Synthetic: 0.4703, Percent Difference: 0.34%
Age - Original: 33.2409, Synthetic: 33.2985, Percent Difference: 0.17%
Outcome - Original: 0.3490, Synthetic: 0.3490, Percent Difference: 0.00%

STD:
Pregnancies - Original: 3.3696, Synthetic: 3.3737, Percent Difference: 0.12%
Glucose - Original: 31.9726, Synthetic: 31.9484, Percent Difference: 0.08%
BloodPressure - Original: 19.3558, Synthetic: 19.3886, Percent Difference: 0.17%
SkinThickness - Original: 15.9522, Synthetic: 15.9914, Per

/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/905815949.py:95: RuntimeWarning: divide by zero encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100
/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/905815949.py:95: RuntimeWarning: invalid value encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100


In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp

# Load the dataset
data = pd.read_csv('diabetes.csv')

# Separate features and target
features = data.drop(columns=['Outcome'])
target = data['Outcome']

# Normalize the features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(features)

# Convert to torch tensors
X = torch.tensor(normalized_features, dtype=torch.float32)
y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)

# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        return self.network(x)

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

input_dim = X.shape[1]
latent_dim = 20  # Size of the noise vector

generator = Generator(latent_dim, input_dim)
discriminator = Discriminator(input_dim)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss function
adversarial_loss = nn.BCELoss()

# Training the GAN
num_epochs = 1000
batch_size = 64

for epoch in range(num_epochs):
    permutation = torch.randperm(X.size()[0])
    for i in range(0, X.size()[0], batch_size):
        # Train Discriminator
        optimizer_D.zero_grad()
        
        real_data = X[permutation[i:i + batch_size]]
        real_labels = torch.ones((real_data.size(0), 1))
        fake_labels = torch.zeros((real_data.size(0), 1))
        
        noise = torch.randn((real_data.size(0), latent_dim))
        fake_data = generator(noise)
        
        real_loss = adversarial_loss(discriminator(real_data), real_labels)
        fake_loss = adversarial_loss(discriminator(fake_data.detach()), fake_labels)
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        
        noise = torch.randn((real_data.size(0), latent_dim))
        fake_data = generator(noise)
        g_loss = adversarial_loss(discriminator(fake_data), real_labels)
        g_loss.backward()
        optimizer_G.step()

    if epoch % 100 == 0:
        print(f"Epoch [{epoch}/{num_epochs}] Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}")

# Sample synthetic data
num_samples = len(data)
noise = torch.randn((num_samples, latent_dim))
with torch.no_grad():
    synthetic_data = generator(noise).numpy()

# Inverse transform the normalized features
synthetic_features = scaler.inverse_transform(synthetic_data)
synthetic_df = pd.DataFrame(synthetic_features, columns=features.columns)
synthetic_df['Outcome'] = target.values

# Statistical comparison
def compare_statistics(original, synthetic):
    stats = {
        "mean": {},
        "std": {},
        "min": {},
        "max": {},
    }

    for column in original.columns:
        orig_stats = original[column].describe()
        synth_stats = synthetic[column].describe()

        stats["mean"][column] = (orig_stats["mean"], synth_stats["mean"])
        stats["std"][column] = (orig_stats["std"], synth_stats["std"])
        stats["min"][column] = (orig_stats["min"], synth_stats["min"])
        stats["max"][column] = (orig_stats["max"], synth_stats["max"])

    return stats

def percent_difference(orig_val, synth_val):
    return abs(orig_val - synth_val) / orig_val * 100

# Compute statistics
stats = compare_statistics(data, synthetic_df)

# Print results and percent differences
for stat_type, columns in stats.items():
    print(f"\n{stat_type.upper()}:")
    for column, values in columns.items():
        orig_val, synth_val = values
        diff = percent_difference(orig_val, synth_val)
        print(f"{column} - Original: {orig_val:.4f}, Synthetic: {synth_val:.4f}, Percent Difference: {diff:.2f}%")

# Kolmogorov-Smirnov Test
print("\nKOLMOGOROV-SMIRNOV TEST:")
for column in features.columns:
    stat, p_value = ks_2samp(data[column], synthetic_df[column])
    print(f"{column} - K-S Statistic: {stat:.4f}, P-value: {p_value:.4f}")


Epoch [0/1000] Discriminator Loss: 1.3351, Generator Loss: 0.6770
Epoch [100/1000] Discriminator Loss: 0.8685, Generator Loss: 0.9934
Epoch [200/1000] Discriminator Loss: 0.7726, Generator Loss: 1.6361
Epoch [300/1000] Discriminator Loss: 0.6548, Generator Loss: 1.4359
Epoch [400/1000] Discriminator Loss: 0.6083, Generator Loss: 1.8280
Epoch [500/1000] Discriminator Loss: 0.7406, Generator Loss: 1.4959
Epoch [600/1000] Discriminator Loss: 1.0855, Generator Loss: 1.5295
Epoch [700/1000] Discriminator Loss: 1.0166, Generator Loss: 1.3697
Epoch [800/1000] Discriminator Loss: 1.2088, Generator Loss: 0.9091
Epoch [900/1000] Discriminator Loss: 1.3762, Generator Loss: 0.9508

MEAN:
Pregnancies - Original: 3.8451, Synthetic: 3.7287, Percent Difference: 3.03%
Glucose - Original: 120.8945, Synthetic: 106.3909, Percent Difference: 12.00%
BloodPressure - Original: 69.1055, Synthetic: 67.7193, Percent Difference: 2.01%
SkinThickness - Original: 20.5365, Synthetic: 21.0729, Percent Difference: 2.61

/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/656960758.py:135: RuntimeWarning: divide by zero encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100
/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_7642/656960758.py:135: RuntimeWarning: invalid value encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100


### Arbitrary dataset script below

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp

class GAN:
    def __init__(self, input_dim, latent_dim=20, lr=0.0002):
        self.latent_dim = latent_dim
        self.input_dim = input_dim
        
        self.generator = self.Generator(input_dim, latent_dim)
        self.discriminator = self.Discriminator(input_dim)
        
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr=lr)
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr=lr)
        self.adversarial_loss = nn.BCELoss()

    class Generator(nn.Module):
        def __init__(self, input_dim, latent_dim):
            super(GAN.Generator, self).__init__()
            self.network = nn.Sequential(
                nn.Linear(latent_dim, 128),
                nn.ReLU(),
                nn.Linear(128, 128),
                nn.ReLU(),
                nn.Linear(128, input_dim)
            )

        def forward(self, x):
            return self.network(x)

    class Discriminator(nn.Module):
        def __init__(self, input_dim):
            super(GAN.Discriminator, self).__init__()
            self.network = nn.Sequential(
                nn.Linear(input_dim, 128),
                nn.ReLU(),
                nn.Linear(128, 128),
                nn.ReLU(),
                nn.Linear(128, 1),
                nn.Sigmoid()
            )

        def forward(self, x):
            return self.network(x)

    def train(self, X, num_epochs=1000, batch_size=64):
        for epoch in range(num_epochs):
            permutation = torch.randperm(X.size()[0])
            for i in range(0, X.size()[0], batch_size):
                # Train Discriminator
                self.optimizer_D.zero_grad()
                
                real_data = X[permutation[i:i + batch_size]]
                real_labels = torch.ones((real_data.size(0), 1))
                fake_labels = torch.zeros((real_data.size(0), 1))
                
                noise = torch.randn((real_data.size(0), self.latent_dim))
                fake_data = self.generator(noise)
                
                real_loss = self.adversarial_loss(self.discriminator(real_data), real_labels)
                fake_loss = self.adversarial_loss(self.discriminator(fake_data.detach()), fake_labels)
                d_loss = real_loss + fake_loss
                d_loss.backward()
                self.optimizer_D.step()

                # Train Generator
                self.optimizer_G.zero_grad()
                
                noise = torch.randn((real_data.size(0), self.latent_dim))
                fake_data = self.generator(noise)
                g_loss = self.adversarial_loss(self.discriminator(fake_data), real_labels)
                g_loss.backward()
                self.optimizer_G.step()

            if epoch % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}")

    def sample(self, num_samples):
        noise = torch.randn((num_samples, self.latent_dim))
        with torch.no_grad():
            return self.generator(noise).numpy()

def compare_statistics(original, synthetic):
    stats = {
        "mean": {},
        "std": {},
        "min": {},
        "max": {},
    }

    for column in original.columns:
        orig_stats = original[column].describe()
        synth_stats = synthetic[column].describe()

        stats["mean"][column] = (orig_stats["mean"], synth_stats["mean"])
        stats["std"][column] = (orig_stats["std"], synth_stats["std"])
        stats["min"][column] = (orig_stats["min"], synth_stats["min"])
        stats["max"][column] = (orig_stats["max"], synth_stats["max"])

    return stats

def percent_difference(orig_val, synth_val):
    return abs(orig_val - synth_val) / orig_val * 100

def perform_ks_test(original, synthetic):
    ks_results = {}
    for column in original.columns:
        stat, p_value = ks_2samp(original[column], synthetic[column])
        ks_results[column] = (stat, p_value)
    return ks_results

def produce_synthetic_data(data, num_epochs=1000, batch_size=64, latent_dim=20):
    # Separate features and target if 'Outcome' column exists
    if 'Outcome' in data.columns:
        features = data.drop(columns=['Outcome'])
        target = data['Outcome']
    else:
        features = data
        target = None

    # Normalize the features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)

    # Convert to torch tensors
    X = torch.tensor(normalized_features, dtype=torch.float32)

    # Initialize and train GAN
    gan = GAN(input_dim=X.shape[1], latent_dim=latent_dim)
    gan.train(X, num_epochs=num_epochs, batch_size=batch_size)

    # Sample synthetic data
    synthetic_data = gan.sample(len(data))

    # Inverse transform the normalized features
    synthetic_features = scaler.inverse_transform(synthetic_data)
    synthetic_df = pd.DataFrame(synthetic_features, columns=features.columns)
    if target is not None:
        synthetic_df['Outcome'] = target.values

    # Statistical comparison
    stats = compare_statistics(data, synthetic_df)

    # Print results and percent differences
    for stat_type, columns in stats.items():
        print(f"\n{stat_type.upper()}:")
        for column, values in columns.items():
            orig_val, synth_val = values
            diff = percent_difference(orig_val, synth_val)
            print(f"{column} - Original: {orig_val:.4f}, Synthetic: {synth_val:.4f}, Percent Difference: {diff:.2f}%")

    # Kolmogorov-Smirnov Test
    print("\nKOLMOGOROV-SMIRNOV TEST:")
    ks_results = perform_ks_test(data, synthetic_df)
    for column, (stat, p_value) in ks_results.items():
        print(f"{column} - K-S Statistic: {stat:.4f}, P-value: {p_value:.4f}")

    return synthetic_df



### Runing on an arbitrary dataset below

In [8]:
data = pd.read_csv('diabetes.csv')
synthetic_data = produce_synthetic_data(data, num_epochs=1000, batch_size=64, latent_dim=20)



Epoch [0/1000] Discriminator Loss: 1.3730, Generator Loss: 0.6557
Epoch [100/1000] Discriminator Loss: 1.1334, Generator Loss: 0.6959
Epoch [200/1000] Discriminator Loss: 1.0905, Generator Loss: 0.9412
Epoch [300/1000] Discriminator Loss: 1.1402, Generator Loss: 1.0711
Epoch [400/1000] Discriminator Loss: 1.1135, Generator Loss: 1.0673
Epoch [500/1000] Discriminator Loss: 1.2576, Generator Loss: 0.9438
Epoch [600/1000] Discriminator Loss: 1.3259, Generator Loss: 0.6600
Epoch [700/1000] Discriminator Loss: 0.9721, Generator Loss: 1.0521
Epoch [800/1000] Discriminator Loss: 1.2036, Generator Loss: 0.8459
Epoch [900/1000] Discriminator Loss: 1.3629, Generator Loss: 0.9694

MEAN:
Pregnancies - Original: 3.8451, Synthetic: -2.0452, Percent Difference: 153.19%
Glucose - Original: 120.8945, Synthetic: 125.3675, Percent Difference: 3.70%
BloodPressure - Original: 69.1055, Synthetic: 94.9516, Percent Difference: 37.40%
SkinThickness - Original: 20.5365, Synthetic: 18.6549, Percent Difference: 9

/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_876/1424367128.py:107: RuntimeWarning: divide by zero encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100
/var/folders/gk/8dd7zppx4vdg26x_p2w7pkn40000gn/T/ipykernel_876/1424367128.py:107: RuntimeWarning: invalid value encountered in scalar divide
  return abs(orig_val - synth_val) / orig_val * 100


# This is Preprocessing the data

In [1]:
from preprocess import drop_invalid_zeros
import pandas as pd

# Placeholder path for the CSV file
csv_path = 'diabetes.csv'

# Load the original dataset for detection of indices
original = pd.read_csv(csv_path)

original_set = drop_invalid_zeros(original)

print(len(original_set))


336


# THIS IS TESTING THE DATA FROM THE SCRIPT

In [2]:


from GAN_Architecture import train_GAN, Sample_Synthetic_Data, CalculateKS





# Train the GAN using the CSV file
generator = train_GAN(original_set)
# Sample synthetic data from the trained generator



Epoch [0/10000] | D Loss: 12.9826 | G Loss: 0.7516
Epoch [100/10000] | D Loss: 0.5902 | G Loss: 0.8320
Epoch [200/10000] | D Loss: 0.4013 | G Loss: 1.1778
Epoch [300/10000] | D Loss: 0.3037 | G Loss: 1.4885
Epoch [400/10000] | D Loss: 0.2969 | G Loss: 1.8221
Epoch [500/10000] | D Loss: 0.1334 | G Loss: 2.3834
Epoch [600/10000] | D Loss: 0.1278 | G Loss: 2.5706
Epoch [700/10000] | D Loss: 0.2035 | G Loss: 2.4562
Epoch [800/10000] | D Loss: 0.1488 | G Loss: 2.6397
Epoch [900/10000] | D Loss: 0.2139 | G Loss: 2.6655
Epoch [1000/10000] | D Loss: 0.3020 | G Loss: 2.9195
Epoch [1100/10000] | D Loss: 0.1039 | G Loss: 3.4978
Epoch [1200/10000] | D Loss: 0.4738 | G Loss: 2.7603
Epoch [1300/10000] | D Loss: 0.2658 | G Loss: 3.3404
Epoch [1400/10000] | D Loss: 1.0131 | G Loss: 2.4546
Epoch [1500/10000] | D Loss: 0.7587 | G Loss: 1.8092
Epoch [1600/10000] | D Loss: 0.5085 | G Loss: 2.4381
Epoch [1700/10000] | D Loss: 0.4758 | G Loss: 2.7024
Epoch [1800/10000] | D Loss: 0.3856 | G Loss: 3.0196
Epoc

In [6]:
num_samples = 336  # Specify the number of samples you want to generate

latent_dim = generator.model[0].in_features  # Extract latent dimension from generator
synthetic_data = Sample_Synthetic_Data(generator, num_samples, latent_dim)

In [7]:
synthetic_set = pd.DataFrame(synthetic_data, columns=original_set.columns)

# Calculate KS test p-values
p_values = CalculateKS(original_set, synthetic_set)

# Print the p-values for verification
print("Kolmogorov-Smirnov Test p-values:")
print(p_values)

Kolmogorov-Smirnov Test p-values:
{'Pregnancies': 0.00023951249463972682, 'Glucose': 9.60069543747106e-06, 'BloodPressure': 0.00012232462921451157, 'SkinThickness': 0.027126269279608848, 'Insulin': 0.2671047678553526, 'BMI': 1.295924993150936e-06, 'DiabetesPedigreeFunction': 1.8168377029864056e-18, 'Age': 3.8081723854136984e-08, 'Outcome': 1.5767867419856404e-23}


In [8]:
import pandas as pd
from IPython.display import HTML



# Function to render a scrollable DataFrame
def display_scrollable_dataframe(df, max_height=400):
    """
    Display a scrollable pandas DataFrame in a Jupyter Notebook.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to display.
    max_height (int): The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        max-height: {max_height}px;
        overflow-y: scroll;
        display: inline-block;
    }}
    </style>
    """
    
    html = df.to_html(classes='scrollable-dataframe')
    display(HTML(style + html))

# Display the DataFrame
display_scrollable_dataframe(synthetic_set, max_height=400)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.250329,68.820740,61.731644,11.789831,41.212143,19.330612,0.539290,17.053885,0.058614
1,0.477909,61.336441,49.586208,19.857014,83.561493,23.396233,-0.363853,15.579238,0.026052
2,2.195090,72.292831,65.834343,14.794847,98.234734,27.121115,-1.212707,21.766598,-0.420209
3,5.662980,135.091766,75.773361,20.288935,115.174294,30.890062,1.291963,40.254356,1.101456
4,3.870366,170.670837,84.659081,21.989534,80.119011,35.224125,-1.665321,23.778173,-0.171568
5,2.241613,119.559319,64.428001,17.083977,52.647579,23.734818,-0.140029,21.755125,0.226905
6,5.402974,163.710907,95.123215,43.263020,238.229187,48.214119,0.647730,42.374786,0.908738
7,-2.036818,52.538963,26.917259,16.670734,95.075562,16.223255,-0.572568,16.587057,-0.821022
8,4.623677,105.939713,87.503403,23.492962,73.607071,30.387600,0.714781,27.996876,-1.448582
9,4.946786,160.340652,86.848213,49.231419,158.550644,49.478062,1.117576,32.578537,0.532403


## Sampling for values only in acceptable range

In [9]:
from GAN_Architecture import selective_sample

num_samples = 336  # Specify the number of samples you want to generate

# Assuming the generator is already trained and available
latent_dim = generator.model[0].in_features  # Extract latent dimension from generator
selected_synthetic_data = selective_sample(generator, num_samples, latent_dim, original_set)

In [10]:
# Generate the synthetic data
selected_synthetic_data = selective_sample(generator, num_samples, latent_dim, original_set)

selected_synthetic_data = pd.DataFrame(selected_synthetic_data, columns=original_set.columns)
# Calculate KS test p-values
p_values = CalculateKS(original_set, selected_synthetic_data)

# Convert the p-values dictionary to a DataFrame for columnar representation
p_values_df = pd.DataFrame(list(p_values.items()), columns=['Column', 'KS p-value'])

# Print the p-values for verification
print("Kolmogorov-Smirnov Test p-values:")
print(p_values_df)


Kolmogorov-Smirnov Test p-values:
                     Column    KS p-value
0               Pregnancies  3.320900e-04
1                   Glucose  9.548827e-08
2             BloodPressure  8.522462e-07
3             SkinThickness  1.702725e-02
4                   Insulin  1.043338e-02
5                       BMI  3.617746e-07
6  DiabetesPedigreeFunction  4.433584e-20
7                       Age  1.958394e-06
8                   Outcome  9.984241e-01


In [11]:
display_scrollable_dataframe(selected_synthetic_data, max_height=400)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,8.384535,197.000000,110.000000,28.380070,113.008240,44.416500,0.085000,40.032551,1
1,4.752225,125.025986,110.000000,23.977978,127.222603,34.824406,0.086246,33.010025,0
2,1.000000,91.188530,38.674927,20.497398,123.593140,24.914305,0.929296,21.000000,0
3,3.911759,104.496872,54.800423,16.660975,117.647751,24.577709,0.085000,22.924479,1
4,2.683282,126.040909,73.057129,33.627453,235.458374,31.720219,1.398140,36.066174,0
5,7.363426,173.690109,108.924553,46.674610,297.131958,50.654675,0.085000,66.695442,1
6,1.000000,92.077301,88.000031,19.607395,46.182255,27.782207,1.499533,28.603529,1
7,3.413773,75.370209,65.927986,26.945938,107.735985,30.818577,0.085000,36.689468,0
8,5.987989,197.000000,87.966812,30.750233,369.308472,40.164925,0.468220,29.952574,1
9,1.000000,69.479416,56.066181,7.000000,62.453495,18.200000,1.048590,28.549755,0


### Utility function

In [1]:
import pandas as pd
from IPython.display import HTML



# Function to render a scrollable DataFrame
def display_scrollable_dataframe(df, max_height=400):
    """
    Display a scrollable pandas DataFrame in a Jupyter Notebook.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to display.
    max_height (int): The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        max-height: {max_height}px;
        overflow-y: scroll;
        display: inline-block;
    }}
    </style>
    """
    
    html = df.to_html(classes='scrollable-dataframe')
    display(HTML(style + html))

### Testing preprocessing for determining batch_size, epochs, learning rate

In [2]:
from preprocess import clean_set, analyze_dataset
from GAN_Architecture import train_GAN, CalculateKS, selective_sample
import pandas as pd

# Placeholder path for the CSV file
csv_path = 'diabetes.csv'

# Load the original dataset for detection of indices
original = pd.read_csv(csv_path)

original_set = clean_set(original)

print(len(original_set))

epochs, lr, batch_size, beta1 = analyze_dataset(original_set)

#
#
#
# Train the GAN using the CSV file
generator = train_GAN(original_set, epochs, batch_size, lr, beta1)





336
Epoch [0/1000] | D Loss: 1.4161 | G Loss: 0.6359
Epoch [100/1000] | D Loss: 0.2035 | G Loss: 2.7809
Epoch [200/1000] | D Loss: 0.4453 | G Loss: 2.3810
Epoch [300/1000] | D Loss: 1.5776 | G Loss: 1.8450
Epoch [400/1000] | D Loss: 1.4648 | G Loss: 1.9553
Epoch [500/1000] | D Loss: 1.2432 | G Loss: 0.9560
Epoch [600/1000] | D Loss: 1.2075 | G Loss: 1.1195
Epoch [700/1000] | D Loss: 1.3278 | G Loss: 0.5104
Epoch [800/1000] | D Loss: 1.3888 | G Loss: 0.5090
Epoch [900/1000] | D Loss: 1.3937 | G Loss: 0.8825


In [4]:
num_samples = 336  # Specify the number of samples you want to generate

# Assuming the generator is already trained and available
latent_dim = generator.model[0].in_features  # Extract latent dimension from generator
selected_synthetic_data = selective_sample(generator, num_samples, latent_dim, original_set)

selected_synthetic_data = pd.DataFrame(selected_synthetic_data, columns=original_set.columns)
# Calculate KS test p-values
p_values = CalculateKS(original_set, selected_synthetic_data)

# Convert the p-values dictionary to a DataFrame for columnar representation
p_values_df = pd.DataFrame(list(p_values.items()), columns=['Column', 'KS p-value'])

# Print the p-values for verification
print("Kolmogorov-Smirnov Test p-values:")
print(p_values_df)

display_scrollable_dataframe(selected_synthetic_data, max_height=400)

Kolmogorov-Smirnov Test p-values:
                     Column    KS p-value
0               Pregnancies  5.569963e-07
1                   Glucose  2.671048e-01
2             BloodPressure  2.633089e-10
3             SkinThickness  3.643383e-03
4                   Insulin  6.240471e-03
5                       BMI  1.295925e-06
6  DiabetesPedigreeFunction  6.287346e-17
7                       Age  6.240471e-03
8                   Outcome  9.938941e-01


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.000000,97.097076,56.971806,28.329041,119.387413,31.683418,0.422339,21.000000,0
1,1.798794,115.731453,55.903606,26.483402,93.777718,32.985443,0.118465,29.053768,1
2,3.347377,160.886505,53.492184,31.933296,171.083664,37.988464,0.085000,40.426968,1
3,1.000000,86.800713,40.859573,19.124380,130.463043,25.144203,1.187088,21.000000,0
4,3.813842,155.395920,79.130852,11.992340,223.713135,37.143040,1.598097,39.879234,0
5,3.707112,133.224503,72.365829,41.667755,38.005917,42.119061,0.085000,32.934891,0
6,1.000000,102.187111,47.905964,14.664574,244.078506,27.600821,2.093288,21.000000,0
7,6.849030,197.000000,110.000000,52.000000,193.991852,57.300000,0.085000,68.041008,1
8,1.000000,89.436447,60.090195,14.985869,95.484627,26.384710,1.267940,21.000000,0
9,1.000000,110.449570,42.030918,21.448410,101.273346,27.985964,0.085000,21.439375,0


## Testing model class

In [2]:
# Import necessary modules
import pandas as pd
from model import Model

# Define the path to the CSV file and the number of samples to generate
csv_path = 'diabetes.csv'
num_samples = 336

# Instantiate the Model class
model = Model(csv_path, num_samples)

# Display the first few rows of the original dataset
print("\nOriginal Data")
display_scrollable_dataframe(model.original_set)

# Display the first few rows of the generated synthetic data
print("\nSelected Synthetic Data:")
display_scrollable_dataframe(model.selected_synthetic_data)

# Display the KS test p-values
print("\nKolmogorov-Smirnov Test p-values:")
print(model.p_values_df)


Epoch [0/1000] | D Loss: 0.9226 | G Loss: 0.6084
Epoch [100/1000] | D Loss: 0.2599 | G Loss: 2.2977
Epoch [200/1000] | D Loss: 0.9025 | G Loss: 4.1096
Epoch [300/1000] | D Loss: 0.6323 | G Loss: 0.7745
Epoch [400/1000] | D Loss: 1.1083 | G Loss: 1.2646
Epoch [500/1000] | D Loss: 1.6777 | G Loss: 0.3500
Epoch [600/1000] | D Loss: 0.8339 | G Loss: 1.1909
Epoch [700/1000] | D Loss: 1.1617 | G Loss: 1.3790
Epoch [800/1000] | D Loss: 1.2473 | G Loss: 1.1508
Epoch [900/1000] | D Loss: 1.2815 | G Loss: 0.5664

Original Data


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
3,1,89,66,23,94,28.1,0.167,21,0
6,3,78,50,32,88,31.0,0.248,26,1
8,2,197,70,45,543,30.5,0.158,53,1
13,1,189,60,23,846,30.1,0.398,59,1
14,5,166,72,19,175,25.8,0.587,51,1
18,1,103,30,38,83,43.3,0.183,33,0
19,1,115,70,30,96,34.6,0.529,32,1
20,3,126,88,41,235,39.3,0.704,27,0
24,11,143,94,33,146,36.6,0.254,51,1
25,10,125,70,26,115,31.1,0.205,41,1



Selected Synthetic Data:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,4.379714,124.480682,59.997177,24.109137,165.044052,23.761999,0.085000,21.000000,1
1,4.994784,93.898132,57.992176,17.894730,99.832710,19.261099,0.085000,21.000000,0
2,3.623966,108.619911,99.528130,22.166435,107.677902,27.213650,1.761202,28.109148,1
3,2.776037,102.940796,49.877510,17.953856,155.305832,18.200000,0.333194,21.000000,0
4,1.813647,103.510139,62.230583,22.763010,140.247101,22.093124,0.881818,21.000000,1
5,1.000000,82.515053,62.958496,19.682999,88.443352,20.474947,0.085000,21.000000,1
6,11.486031,164.165787,100.406471,28.427221,133.636642,30.878931,0.767456,33.678131,1
7,6.285717,173.634705,107.142464,42.168221,217.283646,42.958549,2.042204,30.072590,1
8,5.966463,103.316177,82.476318,19.695990,63.420574,22.947323,0.999658,24.616594,1
9,2.161924,118.568359,75.131447,27.037020,125.799393,27.062035,0.986526,21.000000,1



Kolmogorov-Smirnov Test p-values:
                     Column KS p-value
0               Pregnancies     0.0000
1                   Glucose     0.0000
2             BloodPressure     0.0000
3             SkinThickness     0.0000
4                   Insulin     0.0339
5                       BMI     0.0000
6  DiabetesPedigreeFunction     0.0000
7                       Age     0.0000
8                   Outcome     0.0000


# Start here for testing the MODEL class: 7/15/24 and Newer

In [1]:
import pandas as pd
from IPython.display import HTML



# Function to render a scrollable DataFrame
def display_scrollable_dataframe(df, max_height=400):
    """
    Display a scrollable pandas DataFrame in a Jupyter Notebook.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to display.
    max_height (int): The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        max-height: {max_height}px;
        overflow-y: scroll;
        display: inline-block;
    }}
    </style>
    """
    
    html = df.to_html(classes='scrollable-dataframe')
    display(HTML(style + html))



# Import necessary modules
import pandas as pd
from model import Model

# Define the path to the CSV file and the number of samples to generate
csv_path = 'diabetes.csv'
num_samples = 336

# Instantiate the Model class
model = Model(csv_path, num_samples)

# Display the first few rows of the original dataset
print("\nOriginal Data")
display_scrollable_dataframe(model.original_set)

# Display the first few rows of the generated synthetic data
print("\nSelected Synthetic Data:")
display_scrollable_dataframe(model.selected_synthetic_data)

# Display the KS test p-values
print("\nKolmogorov-Smirnov Test p-values:")
print(model.p_values_df)

Epoch [0/1000] | D Loss: 0.8505 | G Loss: 0.6849
Epoch [100/1000] | D Loss: 0.2964 | G Loss: 2.9930
Epoch [200/1000] | D Loss: 0.5467 | G Loss: 2.0458
Epoch [300/1000] | D Loss: 0.0947 | G Loss: 3.3542
Epoch [400/1000] | D Loss: 0.3044 | G Loss: 2.9140
Epoch [500/1000] | D Loss: 0.7231 | G Loss: 1.7893
Epoch [600/1000] | D Loss: 0.5188 | G Loss: 2.0390
Epoch [700/1000] | D Loss: 0.5953 | G Loss: 1.5926
Epoch [800/1000] | D Loss: 0.9106 | G Loss: 1.6011
Epoch [900/1000] | D Loss: 1.0244 | G Loss: 1.0455

Original Data


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
3,1,89,66,23,94,28.1,0.167,21,0
6,3,78,50,32,88,31.0,0.248,26,1
8,2,197,70,45,543,30.5,0.158,53,1
13,1,189,60,23,846,30.1,0.398,59,1
14,5,166,72,19,175,25.8,0.587,51,1
18,1,103,30,38,83,43.3,0.183,33,0
19,1,115,70,30,96,34.6,0.529,32,1
20,3,126,88,41,235,39.3,0.704,27,0
24,11,143,94,33,146,36.6,0.254,51,1
25,10,125,70,26,115,31.1,0.205,41,1



Selected Synthetic Data:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1.000000,155.176941,56.868900,14.769729,144.761246,28.481689,0.085,30.851658,0
1,3.098751,58.649174,34.209126,9.003386,150.353912,18.200000,0.085,21.000000,0
2,4.306130,159.702042,80.154137,21.951534,212.153091,36.307308,0.085,35.572079,0
3,1.724651,112.311440,55.007702,14.229104,128.216675,24.834360,0.085,24.639353,0
4,3.951046,128.150833,107.231583,40.247696,216.653412,46.330307,0.085,40.936737,0
5,3.610171,108.339890,74.378723,24.947206,173.724014,32.869415,0.085,29.559973,0
6,4.984931,86.408173,55.472042,15.553715,199.819733,23.457146,0.085,21.000000,0
7,1.000000,125.381538,101.942299,32.382919,43.994568,42.126831,0.085,40.407574,0
8,5.530346,127.181496,55.153008,14.540933,222.005692,25.831362,0.085,24.642057,0
9,1.000000,98.784195,63.892841,17.123266,87.254936,26.714039,0.085,25.892351,0



Kolmogorov-Smirnov Test p-values:
                     Column KS p-value
0               Pregnancies   0.000000
1                   Glucose   0.000240
2             BloodPressure   0.000000
3             SkinThickness   0.000000
4                   Insulin   0.002076
5                       BMI   0.000000
6  DiabetesPedigreeFunction   0.000000
7                       Age   0.000000
8                   Outcome   0.000000
